In [12]:
import torch
import numpy as np
from preprocessing import *
from utils import pad_HR_adj, unpad
from model import GSRNet
from dataclasses import dataclass
from hyperparams import Hyperparams

hps = Hyperparams()

In [13]:

model = GSRNet(hps)
model.load_state_dict(torch.load('models/GSR_resnet.pth', map_location=torch.device('cpu')))

model.eval()
unseen_test = get_unseen_test_dset()
preds_list = []

for lr_adj in unseen_test:
    all_zeros_lr = not np.any(lr_adj)

    if all_zeros_lr == False: #choose representative subject
        lr_adj = torch.from_numpy(lr_adj).type(torch.FloatTensor)
        preds,a,b,c = model(lr_adj)
        preds = unpad(preds, hps.padding)

    preds_list.append(preds.detach().numpy())

In [14]:
vectorizer = MatrixVectorizer()

res = []

for hr in preds_list:
    res.append(vectorizer.vectorize(hr))

In [15]:
flattened_array = np.concatenate(res)
df = pd.DataFrame({
    'ID': np.arange(1, len(flattened_array) + 1),
    'Predicted': flattened_array
})

df.to_csv('submission_resnet.csv', index=False)